In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\cedar_signet'

In [3]:
datalist = [x for x in os.listdir(data_folder) if not x.endswith('.csv')]

In [4]:
rc=0 # row counter
for data in tqdm.tqdm(datalist, ascii=True):
    # print(fakeness)
    user_id = int(data.split('.')[0].split('_')[-1])
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        rc+=1
print("The .csv should contain", rc, 'rows!')

100%|##############################################################################| 110/110 [00:00<00:00, 1869.38it/s]


The .csv should contain 2640 rows!


In [5]:
data_f = pd.DataFrame(np.zeros((rc,3)),columns=['user_id','sig_id','fakeness'])
visual_f = pd.DataFrame(np.zeros((rc,2048)))

In [6]:
rc=0 # row counter
for data in tqdm.tqdm(datalist, ascii=True):
    if not(data.endswith('.mat')): continue
    if (data.split('.')[0].split('_')[0] == 'forg'): fakeness = 1
    else: fakeness = 0
    # print(fakeness)
    user_id = int(data.split('.')[0].split('_')[-1])
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        data_f.iloc[rc]['user_id'] = user_id
        data_f.iloc[rc]['sig_id'] = int(row+1)
        data_f.iloc[rc]['fakeness'] = fakeness
        visual_f.iloc[rc] = np.expand_dims(matfile['features'][row], 1).T
        rc+=1

100%|################################################################################| 110/110 [00:01<00:00, 69.54it/s]


In [7]:
# data_f = data_f.reset_index().drop(columns=['index'])
# visual_f = visual_f.reset_index().drop(columns=['index'])

In [8]:
visual_f.shape

(2640, 2048)

In [9]:
data_f.shape

(2640, 3)

In [10]:
data_f.to_csv(os.path.join(data_folder,'data_features.csv'), index=False)
visual_f.to_csv(os.path.join(data_folder,'visual_features.csv'), index=False)